# "FASTAI Generate Wiki-IMDB Language model"
> "Fastpages blog"

- toc:true- branch: master
- badges: true
- comments: true
- author: Conwyn
- categories: [fastai,language,encoder]

In [ ]:
%matplotlib inline
%reload_ext autoreload

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai import *
from fastai.text.all import *

Download the FastAI IMDB entries

In [ ]:
path=untar_data(URLs.IMDB) #/root/.fastai/data/imdb
get_imdb = partial(get_text_files,folders=['train','test','unsup'])

Build a datablock

In [ ]:
dls_lm = DataBlock(
   blocks = TextBlock.from_folder(path,is_lm=True),
    get_items=get_imdb,splitter=RandomSplitter(0.1)
).dataloaders(path,path=path,bs=128,seq_len=80)

Save the Datablock for later when we require the word-token mapping when we use the encoder


In [ ]:
import pickle
pickle.dump( dls_lm , open( "savelm3.p", "wb" ) )
!cp /content/savelm3.p /content/gdrive/MyDrive

In [ ]:
#hide
import pickle
!cp  /content/gdrive/MyDrive/savelm3.p /content
dls_lm = pickle.load( open( "/content/savelm3.p", "rb" ) )
#learnlm = language_model_learner(dls_lm,AWD_LSTM,drop_mult=0.3,metrics=[accuracy,Perplexity()]).to_fp16()

Create the learner

In [ ]:
learnlm = language_model_learner(dls_lm,AWD_LSTM,drop_mult=0.3,metrics=[accuracy, Perplexity()]).to_fp16()

Fit the learner and optionally save the model

In [ ]:
learnlm.fit_one_cycle(1,2e-2)

learnlm.save('1epochF')

!cp /root/.fastai/data/imdb/models/1epochE.pth /content/gdrive/MyDrive

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.015330,3.898061,0.301384,49.306740,26:15


cp: cannot stat '/root/.fastai/data/imdb/models/1epochE.pth': No such file or directory


Note it uses a model subfolder within the path

In [ ]:
print(path)

/root/.fastai/data/imdb


Unfreeze the model and fit. Note this took over five hours. Save the encoder for later

In [ ]:
learnlm.unfreeze()
learnlm.fit_one_cycle(10,2e-3) # 5 hours

learnlm.save_encoder('finetunedF')
!cp /root/.fastai/data/imdb/models/finetunedF.pth /content/gdrive/MyDrive
#learnlm.load_encoder('finetunedF')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.763633,3.761119,0.317067,42.996529,31:10
1,3.701758,3.701771,0.323646,40.518990,30:32
2,3.633814,3.649657,0.329085,38.461483,30:27
3,3.539764,3.616908,0.332909,37.222294,30:55
4,3.499323,3.596352,0.335529,36.464954,30:44
5,3.433929,3.583806,0.337819,36.010345,30:32
6,3.377377,3.572593,0.339740,35.608814,30:54
7,3.294219,3.568537,0.340728,35.464653,30:20
8,3.233820,3.572530,0.341071,35.606556,30:22
9,3.214494,3.576734,0.340802,35.756580,30:31
